In [3]:
from transformers import BlipProcessor, BlipForQuestionAnswering
import requests
from PIL import Image
import json, os, csv
import logging
from tqdm import tqdm
import torch


In [4]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Model/blip-saved-model").to("cuda")

Some weights of BlipForQuestionAnswering were not initialized from the model checkpoint at Model/blip-saved-model and are newly initialized: ['text_encoder.encoder.layer.5.crossattention.self.value.weight', 'text_decoder.bert.encoder.layer.3.attention.self.value.bias', 'text_decoder.bert.encoder.layer.11.crossattention.self.query.bias', 'text_encoder.encoder.layer.5.output.LayerNorm.weight', 'text_encoder.encoder.layer.4.crossattention.output.LayerNorm.bias', 'text_encoder.encoder.layer.4.crossattention.output.LayerNorm.weight', 'text_encoder.encoder.layer.2.attention.output.LayerNorm.bias', 'text_encoder.encoder.layer.6.attention.output.dense.weight', 'text_encoder.encoder.layer.8.output.LayerNorm.bias', 'vision_model.encoder.layers.10.self_attn.projection.bias', 'text_encoder.encoder.layer.10.crossattention.output.dense.bias', 'text_decoder.bert.encoder.layer.5.attention.self.value.bias', 'text_encoder.encoder.layer.6.crossattention.self.value.bias', 'text_encoder.encoder.layer.5.att

In [6]:
import torch
from PIL import Image
import io
from IPython.display import display
from ipywidgets import widgets

# Assuming 'processor' and 'model' are already defined and loaded
def process_image_and_question(change):
    if upload_button.value:
        # Get the image file
        uploaded_file = next(iter(upload_button.value.values()))
        image_data = uploaded_file['content']
        image = Image.open(io.BytesIO(image_data)).convert("RGB")

        # Display the uploaded image
        display(image)

        # Read the question from the text widget
        question = question_input.value

        # Prepare inputs
        encoding = processor(image, question, return_tensors="pt").to("cuda:0", torch.float16)

        # Generate output from the model
        out = model.generate(**encoding)
        generated_text = processor.decode(out[0], skip_special_tokens=True)

        # Display the generated text
        print("Answer:", generated_text)

# Create widgets
upload_button = widgets.FileUpload(
    accept='image/*',  # Accept image/* means it can accept any type of image
    multiple=False  # Allow only single file upload
)

question_input = widgets.Text(
    description='Question:',
    placeholder='Type your question here...'
)

# Set up event to trigger on file upload
upload_button.observe(process_image_and_question, names='value')

# Display widgets
display(upload_button, question_input)


FileUpload(value=(), accept='image/*', description='Upload')

Text(value='', description='Question:', placeholder='Type your question here...')